## Qualitative Evaluation Example

In [ ]:
%matplotlib inline

import random
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import torch
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()

import dense_correspondence
from dense_correspondence.evaluation.evaluation import *
import dense_correspondence.correspondence_tools.correspondence_plotter as correspondence_plotter
from dense_correspondence.dataset.dense_correspondence_dataset_masked import ImageType

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 
                               'dense_correspondence', 'evaluation', 'evaluation.yaml')
config = utils.getDictFromYamlFilename(config_filename)
default_config = utils.get_defaults_config()

# utils.set_cuda_visible_devices([0])
dce = DenseCorrespondenceEvaluation(config)
DCE = DenseCorrespondenceEvaluation

network_name = "robot_box_3"
dcn = dce.load_network_from_config(network_name)
dataset = dcn.load_training_dataset()
DenseCorrespondenceEvaluation.evaluate_network_qualitative(dcn, dataset=dataset, randomize=True)

In [ ]:
def test(dataset, scene_name, dcn, output_feature_map_folder):
    #create directory
    scene_feature_map_dir = os.path.join(output_feature_map_folder, scene_name)
    if not os.path.exists(scene_feature_map_dir):
        os.mkdir(scene_feature_map_dir)
    img_a_idx = 7
    rgb_a = dataset.get_rgb_image_from_scene_name_and_idx(scene_name, img_a_idx)
    #print (rgb_a)
    rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)
    res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
    print (res_a.shape)
    #print (res_a.dtype)

    feature_map_filename = os.path.join(scene_feature_map_dir, str(img_a_idx)+'_feature.npy')
    np.save(feature_map_filename, res_a)
    #print (feature_map_filename)
        
scene_name = "2018-11-07-17-29-42"
output_feature_map_folder = "/home/purplearrow/code/data_volume/pdc/feature_maps"
# please make sure this folder is created

test(dataset, scene_name, dcn, output_feature_map_folder)

In [ ]:
def generate_feature_and_save(dataset, scene_name, dcn, output_feature_map_folder):
    """ given scene_name and dataset, generate feature map use dcn and save to folder
    """
    #create directory
    scene_feature_map_dir = os.path.join(output_feature_map_folder, scene_name)
    if not os.path.exists(scene_feature_map_dir):
        os.mkdir(scene_feature_map_dir)
    for img_a_idx in dataset.get_pose_data(scene_name).keys():
        rgb_a, _, mask_a, _ = dataset.get_rgbd_mask_pose(scene_name, img_a_idx)
        #mask_a = np.asarray(mask_a)
        rgb_a_tensor = dataset.rgb_image_to_tensor(rgb_a)
        res_a = dcn.forward_single_image_tensor(rgb_a_tensor).data.cpu().numpy()
        #print (res_a.shape)
        #print (res_a.dtype)

        feature_map_filename = os.path.join(scene_feature_map_dir, str(img_a_idx)+'_feature.npy')
        np.save(feature_map_filename, res_a)
            


In [ ]:
def load_feature(output_feature_map_folder, scene_name, img_a_idx):
    """ load saved feature map 
    """
    feature_map_filename = os.path.join(output_feature_map_folder, scene_name, str(img_a_idx)+'_feature.npy')
    read_back = np.load(feature_map_filename)
    #print(read_back.shape)
    #print(read_back.dtype)
    return read_back


In [ ]:
scene_name = "2018-11-07-17-29-42"
output_feature_map_folder = "/home/purplearrow/code/data_volume/pdc/feature_maps"
# please make sure this folder is created

generate_feature_and_save(dataset, scene_name, dcn, output_feature_map_folder)
load_res_a = load_feature(output_feature_map_folder, scene_name, 1)